# Library page views – interaction for exploration

This is another look at the small libraries pageview data, but this time combining multiple visualizations linked by interactive selections!

This is just a taste – there are many more examples and ideas in 

- [Interactive Charts documentation](https://altair-viz.github.io/user_guide/interactions.html)
- [Interactive Charts section](https://altair-viz.github.io/gallery/index.html#interactive-charts) of the example gallery

---

*To preserve the mystery, select from the notebook menus*

`Edit -> Clear All Outputs`

---

## Interactive version of VegaFusion renderer

The output won't get saved along with the notebook

In [1]:
import pandas as pd
import altair as alt
from altair import datum

# avoid MaxRowsError with interactive version
import vegafusion as vf
vf.enable_widget()

vegafusion.enable_widget()

## Read in library web site page views data

and change the timestamp column to `datetime` type

In [2]:
pageviews = pd.read_csv('data/pageviews_2012_small.csv',
                       parse_dates=['timestamp'])

pageviews.head()

,timestamp,visitors,city,region,country,longitude,latitude,lcc_description
0,2012-01-01 16:00:00,1,Montreal,Quebec,Canada,-73.5542,45.5089,Military Science
1,2012-01-01 13:00:00,1,Durham,North Carolina,United States,-78.8986,35.9940,History Of The Americas
2,2012-01-01 10:00:00,1,Edinburgh,Scotland,United Kingdom,-3.1875,55.9502,Social Sciences
3,2012-01-01 18:00:00,1,Plymouth,England,United Kingdom,-4.1427,50.3704,NaN
4,2012-01-01 09:00:00,1,Edinburgh,Scotland,United Kingdom,-3.1875,55.9502,Social Sciences


## Simple interactive version of any plot

**Finish any visualization with `.interactive()`**

- Simple zooming with scroll wheel
- Click and drag to pan
- Double-tap to reset zoom range

In [3]:
alt.Chart(pageviews).mark_line().encode(
    x = 'yearmonthdate(timestamp):T',
    y = 'sum(visitors):Q'
).properties(
    width=600,
    height=150
).interactive()

VegaFusionWidget(spec='{\n  "config": {\n    "view": {\n      "continuousWidth": 300,\n      "continuousHeight…

---

## Parameters & Interactive Charts

Documentation: [Parameters & Interactive Charts in Altair](https://altair-viz.github.io/user_guide/interactions.html)

**Look at the documentation to see examples of interactive widgets like sliders linked to parameters!**

---

## Filtering bar chart with time interval selection

**The most exciting part of interactivity comes with defining selections that get shared between multiple plots.** 

Here we create an "interval" selection that only goes in the X direction. 

- The selection is active in the line plot
- It's used as a filter in the bar chart

Reference for "named" colors: [HTML color names](https://www.w3schools.com/tags/ref_colornames.asp)

In [4]:
interval = alt.selection_interval(encodings=['x'])

timeline = alt.Chart(pageviews).mark_line().encode(
    x = alt.X('timestamp:T', timeUnit='yearmonthdate'),
    y = 'sum(visitors):Q',
    color = alt.value("darkslategray")
).properties(
    height = 100,
    width = 500
).add_params(
    interval
)

bar = alt.Chart(pageviews).mark_bar().encode(
    alt.X('sum(visitors):Q'),
    alt.Y('country:N').sort('descending'),
    alt.Color('country:N').legend(None)
).transform_filter(
    interval
)

# Vertical concatenation
timeline & bar

VegaFusionWidget(spec='{\n  "config": {\n    "view": {\n      "continuousWidth": 300,\n      "continuousHeight…

### This interval selection not constrained to x-direction

Conditional color shows which parts of the heatmap have been selected

In [5]:
interval = alt.selection_interval()

heatmap = alt.Chart(pageviews).mark_rect().encode(
    x = 'hours(timestamp):O',
    y = 'day(timestamp):O',
    color=alt.condition(interval, 'sum(visitors)', alt.value('lightgray'))
).add_params(
    interval
)

bar = alt.Chart(pageviews).mark_bar().encode(
    x = 'sum(visitors):Q',
    y = alt.Y('country:N').sort('descending'),
    color = 'country:N'
).transform_filter(
    interval
)

heatmap & bar

VegaFusionWidget(spec='{\n  "config": {\n    "view": {\n      "continuousWidth": 300,\n      "continuousHeight…

## Bars filter timeline with multi-selection

Click on a bar to select. You can hold down shift and select multiple bars and use the combination to filter the timeline.

**Note that we've included conditional color on the bar chart. There is a bug in vega-lite that doesn't allow the same conditional color on the time axis above. There are workarounds, though, like in the [Altair area selection example](https://altair-viz.github.io/gallery/select_mark_area.html)** 

*Notice how different subject areas have different peak times of year for views!*

In [6]:
bar_select = alt.selection_point(empty=True, fields=['lcc_description'])

bar = alt.Chart(pageviews).mark_bar().encode(
    x = 'sum(visitors)',
    y = alt.Y('lcc_description', sort='-x'),
    color=alt.condition(bar_select, alt.value("steelblue"), alt.value("lightgray"))
).properties(
    height = 300
).add_params(
    bar_select
)

timeline = alt.Chart(pageviews).mark_line().encode(
    x = alt.X('timestamp:T', timeUnit='yearmonthdate'),
    y = 'sum(visitors):Q',
    color = alt.value('darkslategray')
).transform_filter(
    bar_select
).properties(
    height = 80
)

bar & timeline

VegaFusionWidget(spec='{\n  "config": {\n    "view": {\n      "continuousWidth": 300,\n      "continuousHeight…

### Selected portion of timeline different color

Requires layering up two timeline plots, one filtered down to just the selected range – that one is colored!

In [12]:
interval = alt.selection_interval(encodings=['x'])

timeline_base = alt.Chart(pageviews).mark_line(
    color = 'gray',
    opacity = 0.3
).encode(
    x = alt.X('timestamp:T', timeUnit='yearmonthdate'),
    y = 'sum(visitors):Q',
).properties(
    height = 80,
    width = 400
).add_params(
    interval
)

timeline_selected = timeline_base.mark_line(color='goldenrod').transform_filter(interval)

bar = alt.Chart(pageviews).mark_bar().encode(
    x = 'sum(visitors)',
    y = alt.Y('lcc_description', sort='-x'),
).properties(
    height = 300,
    width = 400
).transform_filter(
    interval
)

((timeline_base + timeline_selected) & bar).configure_axis(labelLimit=500)

VegaFusionWidget(spec='{\n  "config": {\n    "view": {\n      "continuousWidth": 300,\n      "continuousHeight…

### Keep consistent ordering of LCC categories so more easily see variations over selection

Maybe there's a way to do this `rank()` calculation in Altair with a `.transform_window()`, but it was easier for me to just pre-calculate it in Pandas.

In [13]:
pageviews['lcc_rank'] = (pageviews.groupby('lcc_description')['visitors']
                         .transform('sum')
                         .rank(ascending=False, method='dense')
                        )

interval = alt.selection_interval(encodings=['x'])

timeline_base = alt.Chart(pageviews).mark_line(
    color = 'gray',
    opacity = 0.3
).encode(
    x = alt.X('timestamp:T', timeUnit='yearmonthdate'),
    y = 'sum(visitors):Q',
).properties(
    height = 80,
    width = 400
).add_params(
    interval
)

timeline_selected = timeline_base.mark_line(color='goldenrod').transform_filter(interval)

bar = alt.Chart(pageviews).mark_bar().encode(
    x = 'sum(visitors)',
    y = alt.Y('lcc_description').sort(alt.SortField('lcc_rank')),
).properties(
    height = 300,
    width = 400
).transform_filter(
    interval
)

((timeline_base + timeline_selected) & bar).configure_axis(labelLimit=500)

VegaFusionWidget(spec='{\n  "config": {\n    "view": {\n      "continuousWidth": 300,\n      "continuousHeight…